In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from matplotlib import rcParams
import scanpy as sc
import os
import anndata
from sklearn.mixture import GaussianMixture
from fcsy import DataFrame
import matplotlib
from glob import glob
matplotlib.rcParams['pdf.fonttype']=42
matplotlib.rcParams['ps.fonttype']=42
import warnings
warnings.filterwarnings("ignore")
from igraph import InternalError

# scanpy settings
sc.settings.verbosity = 0  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, frameon=False, figsize=(4, 4)) 
sc._settings.ScanpyConfig.n_jobs=4 # useless

In [2]:
# readin the information table
info_path = 'data/211109_Metadata_GRID_ID.csv'
sampleInfo = pd.read_csv(info_path, index_col=0, dtype={'Visit': str})
sampleInfo = sampleInfo[sampleInfo['Excluded']!='Yes']
sampleInfo = sampleInfo[~sampleInfo['batchGRID'].isna()]

In [3]:
#EXP-20-CL8796, EXP-21-DG3628, EXP-20-DG3622
batch_name = 'DG3622_and_DG3628'
selectInfo = sampleInfo[sampleInfo['batchGRID'].isin(['EXP-20-DG3622', 'EXP-21-DG3628'])]
selectInfo = selectInfo[selectInfo['Transition']=='FtM']
#selectInfo.to_csv('merged_sample_info.csv')

In [4]:
dataDir = '/Users/tan/cytof_data/*/renamed'
labelDir = '/Users/tan/cytof_data/*/classifiedSlim'
nsample = 3000
#sub_name = 'gdT'

drop_columns = ['Time', 'Event_length', '102Pd', '104Pd', '105Pd', '106Pd',
                '108Pd', '116Cd', '131Xe', '133Cs', '140Ce', 'DNAIr191',
                'DNAIr193', 'Center', 'Offset', 'Width', 'Residual']
drop_dic = {'pDC': [], 
            'B-cells': ['CD33', 'CD3e', 'gdTCR', 'Siglec-8', 'CD14', 'CD141', 'CD4'],
            'CD4 T-cells': ['IgD', 'CD1c', 'gdTCR', 'Siglec-8', 'CD20', 'CD14'],
            'CD8 T-cells': ['IgD', 'CD11c', 'CD1c', 'gdTCR', 'Siglec-8', 'CD20', 'CD14'],
            #'Eosinophils': ['IgD', 'CD57', 'CD25', 'TCRgd', 'CD14'],
            'Monocytes': ['CD57', 'IgD', 'CD25', 'CD20', 'gdTCR', 'CD22', 'CD127'], 
            'Neutrophils': ['IgD', 'HLA-DR', 'CD57', 'CD25', 'CD22', 'gdTCR', 'CD123', 'CD161'],
            'NK cells': [],
            'Lin Neg': [],
            'gdT': [], 
            'Plasmablasts': [],
            'Basophils': []}

# readin and merge the file according to "selectInfo" as an anndata object
all_data_path = 'adata/'+ batch_name +'_sex_change_FtM_no_covariates.h5ad'

for sub_name in drop_dic:
    print('currently working on ' + sub_name)
    if not os.path.exists(all_data_path):
        print('adata not found, load and preprocess raw data...')
        all_data_list = []
        all_label_list = []
        for i in range(len(selectInfo)):
            label_path = glob(labelDir + '/*/' + str(selectInfo['GRID_ID'].iloc[i]) + '*.csv')[0]
            data_path = glob(dataDir + '/*/' + str(selectInfo['GRID_ID'].iloc[i]) + '*.fcs')[0]
            labelTmp = pd.read_csv(label_path)
            labelTmp['Sample ID'] = np.repeat(selectInfo['Subject'].iloc[i], len(labelTmp.index))
            labelTmp['timepoint'] = np.repeat(selectInfo['Visit'].iloc[i], len(labelTmp.index))
            labelTmp['Subject ID'] = np.repeat(str(selectInfo['SubjectID'].iloc[i]), len(labelTmp.index))
            labelTmp['group'] = np.repeat(str(selectInfo['Transition'].iloc[i]), len(labelTmp.index))
            labelTmp['timepoint_group'] = np.repeat(selectInfo['Visit'].iloc[i] + 
                                                    '_' + 
                                                    str(selectInfo['Transition'].iloc[i]), len(labelTmp.index))
            labelTmp['batch'] = np.repeat(data_path.split('/')[-4], len(labelTmp.index)) #EXP-XX-XXXXXX
            dataTmp = DataFrame.from_fcs(data_path, channel_type='long')
            if '4-1BB' in set(dataTmp.columns):
                dataTmp.rename(columns={"4-1BB": "CD137"}, inplace=True)
            # filter the cells without a level1 tag
            dataTmp = dataTmp[labelTmp['level1']!=' ']
            labelTmp = labelTmp[labelTmp['level1']!=' ']
            # remove EQBeads and DNA channel # also remove the negative channels
            dataTmp.drop(columns=drop_columns, inplace=True)
            #dataTmp = dataTmp[select_columns]
            dataTmp = np.arcsinh(dataTmp/5)
            all_data_list.append(dataTmp)
            all_label_list.append(labelTmp)

        all_data = pd.concat(all_data_list, ignore_index=True)
        all_label = pd.concat(all_label_list, ignore_index=True)
        adata = anndata.AnnData(all_data)
        adata.obs = all_label
        print('batch correction...')
        #sc.pp.combat(adata, key = 'batch', covariates = ['timepoint'])
        sc.pp.combat(adata, key = 'batch')
        print('scaling...')
        sc.pp.scale(adata)
        os.makedirs('adata/', exist_ok=True)
        print('write to h5ad file...')
        adata.write(filename=all_data_path, compression = 'gzip')
        adata=None
        print('finished!')
    print('loading...')
    adata_all = sc.read_h5ad(filename = all_data_path)
    print('finished!')


    #pd.set_option('display.max_rows', adata_all.obs['Sample ID'].nunique()+1)
    #adata_all.obs[adata_all.obs['level2']==sub_name]['Sample ID'].value_counts()

    # subsampling
    print('subsampling...')
    try:
        if sub_name in adata_all.obs['level1'].unique().tolist():
            sample_index = adata_all.obs[adata_all.obs['level1']==sub_name].groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)
        elif sub_name in adata_all.obs['level2'].unique().tolist():
            sample_index = adata_all.obs[adata_all.obs['level2']==sub_name].groupby('Sample ID').apply(lambda x: x.sample(n=nsample, random_state=0) if x.shape[0]>=nsample else x).index.droplevel(level=0)
        adata_sample = adata_all[sample_index]
    except ValueError as e: 
        # usually when no file has more cells than nsample and thus no subsampling at all.
        # so the results after apply will not be a multiplex index and the droplevel func will fail.
        print(e)
        if sub_name in adata_all.obs['level1'].unique().tolist():
            adata_sample = adata_all[adata_all.obs['level1']==sub_name]
        elif sub_name in adata_all.obs['level2'].unique().tolist():
            adata_sample = adata_all[adata_all.obs['level2']==sub_name]

    adata = anndata.AnnData(adata_sample.to_df().drop(columns = drop_dic[sub_name]))
    adata.obs = pd.DataFrame(adata_sample.obs)

    # skip
    # optimization
    #sc.settings.figdir='./optimization/'
    #for res in [0.3]:
    #    adata_opt = adata
    #    n_comps = min([adata_opt.n_obs, adata_opt.n_vars, 21])-1
    #    sc.tl.pca(adata_opt, svd_solver='arpack', n_comps=n_comps)
    #    sc.pp.neighbors(adata_opt, n_neighbors=10, n_pcs=n_comps)
    #    sc.tl.leiden(adata_opt, resolution=res)
    #    sc.tl.paga(adata_opt, groups='leiden')
    #    sc.pl.paga(adata_opt, color=['leiden'], threshold=0.1, show=False, 
    #               save='_' + sub_name + '_' + str(res) + '.pdf')
    #    adata_opt = []

    # save figures to a sub dir
    figpath='./figures/' + batch_name + '/' + sub_name + '/'
    os.makedirs(figpath, exist_ok=True)
    sc.settings.figdir=figpath

    print('calculating PAGA...')
    #n_comps = min([adata.n_obs, adata.n_vars, 21])-1
    #sc.tl.pca(adata, svd_solver='arpack', n_comps=n_comps)
    sc.pp.neighbors(adata, n_neighbors=10, use_rep='X')

    # paga process
    sc.tl.leiden(adata, resolution=0.3) 

    sc.tl.paga(adata, groups='leiden')
    try:
        sc.pl.paga(adata, color=['leiden'], threshold=0.1, show=False, 
                   save='_' + sub_name + '.pdf')
    except InternalError as e: # maybe there're too little cells
        print(e)
        sc.pl.paga(adata, color=['leiden'], show=False, 
                   save='_' + sub_name + '.pdf')        

    print('embedding with FA...')
    sc.tl.draw_graph(adata, init_pos='paga')

    sc.pl.draw_graph(adata, color=['timepoint', 'group', 'leiden', 'batch', 'Subject ID'], show=False,
                     save='_' + sub_name + '.pdf')

    sc.pl.draw_graph(adata, color=adata.var.index.values, show=False,
                     save='_' + sub_name + '_markers.pdf')

    print('embedding with density plot...')

    #sc.tl.embedding_density(adata, basis='draw_graph_fa', groupby='group')
    #sc.pl.embedding_density(adata, basis='draw_graph_fa', key='draw_graph_fa_density_group', 
    #                        group=['FtM', 'MtF'], show=False, 
    #                        save='_' + sub_name + '_group_density.pdf')
    sc.tl.embedding_density(adata, basis='draw_graph_fa', groupby='timepoint_group')
    sc.pl.embedding_density(adata, basis='draw_graph_fa', key='draw_graph_fa_density_timepoint_group', 
                            group=['1_FtM', '2_FtM', '3_FtM'], show=False, 
                            save='_' + sub_name + '_timepoint_FtM_density.pdf')
    #sc.pl.embedding_density(adata, basis='draw_graph_fa', key='draw_graph_fa_density_timepoint_group', 
    #                        group=['1_MtF', '2_MtF', '3_MtF'], show=False, 
    #                        save='_' + sub_name + '_timepoint_MtF_density.pdf')

    print('saving results...')
    os.makedirs('PAGA_result_data/'+batch_name+'/', exist_ok=True)
    adata.write(filename='PAGA_result_data/'+ batch_name + '/all_' + sub_name + '_sample3000.h5ad', compression = 'gzip')
    print('done!')
    
    

currently working on pDC
adata not found, load and preprocess raw data...
batch correction...


... storing 'level0' as categorical
... storing 'level1' as categorical
... storing 'level2' as categorical
... storing 'Sample ID' as categorical
... storing 'timepoint' as categorical
... storing 'Subject ID' as categorical
... storing 'group' as categorical
... storing 'timepoint_group' as categorical
... storing 'batch' as categorical


scaling...
write to h5ad file...
finished!
loading...
finished!
subsampling...
Cannot remove 1 levels from an index with 1 levels: at least one level must be left.
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
currently working on B-cells
loading...
finished!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
currently working on CD4 T-cells
loading...
finished!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
currently working on CD8 T-cells
loading...
finished!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
currently working on Monocytes
loading...
finished!
subsampling...
calculating PAGA...
embedding with FA...
embedding with density plot...
saving results...
done!
currently working on Neutrophils
loading...
finished!
subsampling...
calculating PAGA.